In [2]:
import pandas as pd

from openpyxl import load_workbook

import numpy as np
import uncertainties as uc

pd.options.display.float_format ='{:,.1f}'.format
import matplotlib.pyplot as plt

%matplotlib inline  

import scipy.stats as stats
from uncertainties import ufloat
from uncertainties import unumpy as unp 
# from uncertainties import unumpy.uarray as uarr

from sklearn.linear_model import LinearRegression

from scipy.stats import gmean
%run Utility_Functions.ipynb

## Methods and Statistics
Ranges are given as 1 standard error of the values (not symmetrical). 


# Liver cells
The main functional cells of the liver are the hepatocytes that compose most of its mass and are responsible for most of its function. In addition to them, in the liver there are several other type of cell with various functions.
The main cells of the liver are:
1. Hepatocytes -cells of the main parenchymal tissue of the liver. Involved in: protein synthesis,protein storage, transformation of carbohydrates, synthesis of cholesterol, bile salts and phospholipids, detoxification, modification, and excretion of exogenous and endogenous substances, initiation of formation and secretion of bile
1. Kupffer cells - tissue resident macrophages
2. Stellate cells - fat storing cells
3. Endothelial cells- lining the blood cells in the liver

In [3]:
cell_dist = pd.read_excel('Liver_cells_data.xlsx','Cell_dist',index_col=0,usecols = range(17),skipfooter=3)

liver = CellTypesResDF(cell_dist.index[0:3],un_type='mul')

## Hepatocytes - numbers and mass
We integrate data regarding 3 parameters to conclude the estimate the number of hepatocytes and their total mass:
1. The volume fraction of the liver made out of hepatocytes (could be acounted as parenchymal cell volume)
2. The volume of a single hepatocyte
3. The density of hepatocytes in 1 gram or 1 ml of liver tissue (reference density given in [Overmoyer et al., 1987](https://www.ncbi.nlm.nih.gov/pubmed/3579513))
From the third parameter we can derive an estiamte for the total number of cells using the total mass of the liver [Snyder,1975](http://www.icrp.org/publication.asp?id=ICRP%20Publication%2023). 
Furthermore, the third parameter is the quotient of the first two, so we use the known values from some of the source to get an estimate for the third parameter for these that miss it ([Wiśniewski et al., 2016](https://www.ncbi.nlm.nih.gov/pubmed/26825538), [Yoshikado et al., 2017](http://dmd.aspetjournals.org/content/dmd/45/7/779.full.pdf?with-ds=yes)).

We assume here that the hepatocytes density is representing for the entire liver, including residual blood etc. This may be causing some overestimation, to take this un account we compare the total cellular mass derived from the given data to the predicted cellular mass derived from the potassium content of the liver, and calibrate the relevant volume fraction factor accordingly.
 
Sources:
[Gates et al., 1961](https://www.ncbi.nlm.nih.gov/pubmed/13703917),
[Rohr et al., 1976](https://link.springer.com/article/10.1007/BF00433072),
[de la Iglesia et al., 1976](https://www.ncbi.nlm.nih.gov/pubmed/1247085),
[Duarte et al., 1989](https://www.ncbi.nlm.nih.gov/pubmed/2752360),
[Immamura et al., 1991](https://aasldpubs.onlinelibrary.wiley.com/doi/abs/10.1002/hep.1840140308),
[Wiśniewski et al., 2016](https://www.ncbi.nlm.nih.gov/pubmed/26825538),
[Yoshikado et al., 2017](http://dmd.aspetjournals.org/content/dmd/45/7/779.full.pdf?with-ds=yes).






In [4]:
hepato = pd.read_excel('Liver_cells_data.xlsx','Human_hepatocytes',index_col=0,usecols = range(10))
liver_propert= pd.read_excel('Liver_cells_data.xlsx','Total_cell_mass_data',index_col=0,usecols = range(8))
liv_den =ufloat(liver_propert.loc['liver density','value'],liver_propert.loc['liver density','SEM'])

liv_mass = ufloat(liver_propert.loc['liver mass','value'],liver_propert.loc['liver mass','SEM'])
hepato.head()


,volume fraction of hepatocytes,SEM volume fraction of hepatocytes,hepatocytes volume [fl],SEM hepatocytes volume [fl],hepatocytes per ml,SEM hepatocytes per ml,hepatocytes per g,SEM hepatocytes per g,comments
source,,,,,,,,,
"Gates et al., 1961",nan,nan,nan,nan,nan,nan,"171,000,000.0","9,974,957.0",NaN
"Wiśniewski et al., 2016",nan,nan,"3,144.7",95.7,nan,nan,nan,nan,NaN
"Rohr et al., 1976",0.8,0.0,"11,300.0",800.0,"102,000,000.0","11,000,000.0",nan,nan,NaN
"Imamura et al., 1991",0.7,0.1,nan,nan,"176,229,508.2","67,413,669.2",nan,nan,"Cirrhotic Patients ,include non-hepatocytes th..."
"Yoshikado et al., 2017",nan,nan,"2,300.0",190.5,nan,nan,nan,nan,NaN


In [5]:
#adding a column of total cells per gram of liver, including the error (as uarray)
hepato['cells/g'] = unp.uarray(hepato['hepatocytes per g'],hepato['SEM hepatocytes per g'])

#for the density given per 1ml we divide br the liver mass density
hepato.loc[~np.isnan(hepato['hepatocytes per ml']), 'cells/g'] =  1/liv_den*unp.uarray(hepato['hepatocytes per ml'],
                                                                             hepato['SEM hepatocytes per ml'])[~np.isnan(hepato['hepatocytes per ml'])]

#computing the average volume fraction of hepatocyte out of the total liver, to enable derivation of estimates for the density where given the mean volume
#of cells, and vice versa
vol_frac = ufloat(np.nanmean(hepato['volume fraction of hepatocytes']),
                  np.nanmean(hepato['SEM volume fraction of hepatocytes'])) + ufloat(0,np.nanstd(hepato['volume fraction of hepatocytes'])/np.sqrt(hepato['volume fraction of hepatocytes'].count()))

#assigning the index of rows where only the mean volume of cells is given. So they can be chosen and the density estimated, base on the mean volume fraction
indx = np.isnan(hepato['hepatocytes per ml']) & ~np.isnan(hepato['hepatocytes volume [fl]']) & np.isnan(hepato['hepatocytes per g'])

#setting the density of heparocytes based on their mean volume
hepato.loc[indx,'cells/g'] =  vol_frac*10**12/(liv_den*unp.uarray(hepato['hepatocytes volume [fl]'],hepato['SEM hepatocytes volume [fl]'])[indx])

#using a utlity function GMeanWithErros that get a uarray vector with uncertainty and estimat the geomtric mean and errors, using converstion to 
# log space, and acounting for both the inter errors that is between the values and the intra error, that is given from the average of the individual errors

print('Before calibraton with total cellular mass data:')
liver.res.loc['hepatocytes','number'] = gmean_with_errors(hepato['cells/g']*liv_mass)
liver.print_params('number','hepatocytes')



Before calibraton with total cellular mass data:
Number of hepatocytes is: 3e+11 cells (SD range: 3e+11 - 4e+11 cells)


C:\Users\ronse\anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


We can use the reference values to get an estimate for the cellular mass and total cellular mass of hepatocytes. We use the volume fraction and density values to fill in the gaps for sources that didn't give estimate fot the mean volume: 

(Some values might be overestimating beacuse the use cells closer to the main blood vessels but it suppose to be balanced by sources that focus on cells in the intermediate lobular zone) 

In [6]:
#for cells with known volume we use the density of the liver to derive an estimate for the cellular mass:
hepato['cell mass [pg]'] =  liv_den*unp.uarray(hepato['hepatocytes volume [fl]'],hepato['SEM hepatocytes volume [fl]'])


#assigning the index of rows where only the mean volume of cells is missing. So they can be chosen and the density estimated, 
#based on the mean volume fraction and density
indx = ~np.isnan(hepato['hepatocytes per ml']) & np.isnan(hepato['hepatocytes volume [fl]']) &  ~np.isnan(hepato['volume fraction of hepatocytes'])

#setting the mean volume using the volumatric density and the total volume fraction given in the source:
hepato.loc[indx,'cell mass [pg]'] = liv_den*(unp.uarray(hepato['volume fraction of hepatocytes'],
                                                        hepato['SEM volume fraction of hepatocytes'])[indx]* 10**12/(unp.uarray(hepato['hepatocytes per ml'],hepato['SEM hepatocytes per ml'])[indx]))

#assigning the index of rows where only both mean volume of cells and the fraction of hepatocytes are missing, so they can be chosen and the density estimated, 
#based on the average volume fraction and the hepaocyte density per g from the source
indx = ~np.isnan(hepato['hepatocytes per g']) & np.isnan(hepato['hepatocytes volume [fl]']) &  np.isnan(hepato['volume fraction of hepatocytes'])

#setting the mean volume using the hepatocyte per g data  the mean total volume fraction estimated earlier given in the source:
hepato.loc[indx,'cell mass [pg]'] = (vol_frac*10**12/(unp.uarray(hepato['hepatocytes per g'],hepato['SEM hepatocytes per g'])[indx]))


#using a utlity function GMeanWithErros that get a uarray vector with uncertainty and estimat the geomtric mean and errors, using converstion to 
# log space, and acounting for both the inter errors that is between the values and the intra error, that is given from the average of the individual errors


liver.res.loc['hepatocytes','cell mass'] = gmean_with_errors(hepato['cell mass [pg]'])
liver.print_params('cell mass','hepatocytes')


#Multiplication of the numebrs of cells and single cell mass give the total mass of hepatocytes:

liver.res.loc['hepatocytes','total cellular mass'] = gmean_with_errors(hepato['cell mass [pg]']*hepato['cells/g']*liv_mass/10**12)

#using function to print
print('Before calibraton with total cellular mass data:')
liver.print_params('total cellular mass','hepatocytes')



Cell mass of hepatocytes is: 4300 pg (SD range: 3400 - 5300 pg)
Before calibraton with total cellular mass data:
Total cellular mass of hepatocytes is: 1460 grams (SD range: 1140 - 1900 grams)


This seems to be an overestimate, as the total cellular mass of the liver is less.

We can use the total cellular mass of the liver to calibrate the results:
As Most of the cellular mass of the liver reside in the hepatocytes, we can use the total cellular mass of the liver derived from  potasium concentraion to asses the fraction of cellular mass based on a furmola from [Wang et al., 2004](https://www.ncbi.nlm.nih.gov/pubmed/14532167) (potassium content taken from [Snyder et al., 1975](http://www.icrp.org/publication.asp?id=ICRP%20Publication%2023))

In [7]:
liv_pot = ufloat(liver_propert.loc['liver potassium','value'],liver_propert.loc['liver potassium','SEM'])
conv_f = ufloat(liver_propert.loc['cell mass/potassium conc.','value'],liver_propert.loc['cell mass/potassium conc.','SEM'])

#following Wang et al., 2004 formula and converting from kg to g twice
liver_cellular_mass = 10**6*liv_pot*conv_f/liver_propert.loc['potassium molar mass','value']

total_hept_mass = liver_cellular_mass*ufloat(liver_propert.loc['volume fraction hepatocyte/total cells','value'],
                                             liver_propert.loc['volume fraction hepatocyte/total cells','SEM'])


liver.res.loc['hepatocytes','total cellular mass'] = round(total_hept_mass.nominal_value,-1),round(ufloat_to_mulfac(total_hept_mass),2)


calibr_hepa_number = 10**12*liver.res.loc['hepatocytes','total cellular mass'][0]/liver.res.loc['hepatocytes','cell mass'][0]
calibr_hepa_num_error = propagate_mul_facs([liver.res.loc['hepatocytes','total cellular mass'][1],
                                            liver.res.loc['hepatocytes','cell mass'][1]])


#using log10 to identefy the significant numbers and the round function to trim them
liver.res.loc['hepatocytes','number'] = calibr_hepa_number, calibr_hepa_num_error


print('After calibraton with total cellular mass data derived from potassium content:')
liver.print_params(['number','total cellular mass'],'hepatocytes')

After calibraton with total cellular mass data derived from potassium content:
Number of hepatocytes is: 2e+11 cells (SD range: 2e+11 - 3e+11 cells)
Total cellular mass of hepatocytes is: 980 grams (SD range: 710 - 1350 grams)


## Kupffer and stellate cells
The dircet measurements of non-parenchymal liver cells (liver cells other than hepatocytes) density in humans is [Gates et al., 1961](https://www.ncbi.nlm.nih.gov/pubmed/13703917), who measured the total cell density of non-parenchymal liver cells, not including endothelial and bile epithelium. There was no seperation to Kupffer and stellate cells.
We base our estimates on their density, using the proportion of the two population, based on measurment in rodents ([Feudenberg et al., 1989](https://www.ncbi.nlm.nih.gov/pubmed/2569248), 
[Bouwen et al., 1986](https://www.ncbi.nlm.nih.gov/pubmed/3733004), 
[Knook and Sleyster, 1976](https://www.ncbi.nlm.nih.gov/pubmed/994558), 
[Baratta et al., 2009](https://www.ncbi.nlm.nih.gov/pubmed/19255771))



In [8]:
Gates_1961 = pd.read_excel('Liver_cells_data.xlsx','Gates et al., 1961',index_col=0,usecols = range(9))
cell_dist_data= pd.read_excel('Liver_cells_data.xlsx','Cell_dist',index_col=0,usecols = range(17),skipfooter=3)


In [9]:
#We want to extract the fraction of Kupffer fraction out of Kupffer and Stellate cells
#first we put in relevant columns the values with their uncertainty

cell_dist_data['[2]'] = cell_dist_data['% of mice  non-parenchymal cells [2]']
cell_dist_data['[3]'] = unp.uarray(cell_dist_data['% of rat  non-parenchymal cells [3]'],cell_dist_data['% SD of rat  non-parenchymal cells [3]'])
cell_dist_data['[5]'] = unp.uarray(cell_dist_data['% of mice hepatic cells [5]'],cell_dist_data['SEM % of mice hepatic cells [5]'])

#then we take this columns and compute the proportion of Kupffer cells out of their sum with stellate cells 
Kupf_set_dist = cell_dist_data[['[2]','[3]','[5]']]
Kupf_fractions = Kupf_set_dist.loc['Kupffer cells'] /(Kupf_set_dist.loc['Kupffer cells']+Kupf_set_dist.loc['stellate cells'])

#Adding the two error terms: intra-error the average of the error from each sourcs, inter error the variation of the nominal values
Kup_frac = Kupf_fractions.sum()/Kupf_fractions.size + ufloat(0,unp.nominal_values(Kupf_fractions).std()/Kupf_fractions.size)

print('The proportion of Kupffer cells out of Littoral cells: {:0.0fP}%'.format(100*Kup_frac))

The proportion of Kupffer cells out of Littoral cells: 65±4%


We use the best estimate taken from the calibration of hepatocytes number, using mass data, to calibrate the total number of littoral cells (stellate and Kupffer cells) in the liver. This way we wish to cancel biases in the measurement of  [Gates et al., 1961](https://www.ncbi.nlm.nih.gov/pubmed/13703917), by using only the ratio of littoral cells to hepatocytes .
Furthemore, we comapre the results to data regarding the prelevance and density of stellate cells from [Giampieri et al., 1981](https://www.karger.com/Article/PDF/198640) and [Sztark et al., 1986](https://www.sciencedirect.com/science/article/pii/S0168827886800472).
We emphisize that the density was mesured in small section in a manner that tend to overestimate.

In [10]:
cell_density = pd.Series(unp.uarray(Gates_1961.loc[['hepatocytes','stellate and Kupffer cells'],'average number of cells per mg'],
                          Gates_1961.loc[['hepatocytes','stellate and Kupffer cells'],'SEM number of cells per mg']),index =['hepatocytes','stellate and Kupffer cells'])

#using the calibrated number of hepatocyte to estiamte the littoral cells number, using the measured ratio of hepatocyte to littoral cells in human
total_littoral_cells = cell_density['stellate and Kupffer cells']*calibr_hepa_number/cell_density['hepatocytes']
#using the utilities function to translate the uncertainty into multiplication factor and to propogate the errors
littoral_cells_unc = propagate_mul_facs([ufloat_to_mulfac(total_littoral_cells),calibr_hepa_num_error]) 
total_littoral_cells = total_littoral_cells.nominal_value

print('Total littoral cells: {:0.1e}'.format(total_littoral_cells),
          'SD range {:0.1e}'.format(total_littoral_cells/littoral_cells_unc),
          '- {:0.1e} cells'.format(total_littoral_cells*littoral_cells_unc))

#using the fracion of kupffer cells out of the littoral cells
Kupf_num = Kup_frac.nominal_value * total_littoral_cells
Kupf_num_unc = propagate_mul_facs([ufloat_to_mulfac(Kup_frac),littoral_cells_unc]) 

#using log10 to identefy the significant numbers and the round function to trim them
liver.res.loc['Kupffer cells','number'] = Kupf_num,Kupf_num_unc

#using the fracion of kupffer cells out of the littoral cells to estimate the stellate cells num
ste_num = (1-Kup_frac.nominal_value)* total_littoral_cells
ste_num_unc = propagate_mul_facs([ufloat_to_mulfac(Kup_frac),littoral_cells_unc]) 

#using log10 to identefy the significant numbers and the round function to trim them
liver.res.loc['stellate cells','number'] = ste_num, ste_num_unc

liver.print_params('number',['Kupffer cells','stellate cells'])


print('_______')

# Sztark et al., 1986 density derived estimate for the number of stellate cells
szt_stell_num = (cell_dist_data.loc['stellate cells','cells per ml [7]']*liv_mass/liv_den).nominal_value
print('Sztark et al., 1986 density derived estimate for the number of stellate cells: {:0.1e}'.format(szt_stell_num))
print('That overstimate might be due to the small section that was taken just from the parenchymal volume, not calibrated as the results derived from Gates et al., 1981')
print('')
#comaprison to Giampieri et al., 1981 data regarding the ratio of stellate cells to hepatocytes:
print('Estimated stellate cell to hepatocyte ratio: {:0.1f}%'.format(100*liver.res.loc['stellate cells',
                                                                                           'number'][0]/liver.res.loc['hepatocytes','number'][0])  )
print('Giampieri et al., 1981 stellate cell to hepatocyte ratio: {:0.1f}%'.format(100*cell_dist_data.loc['stellate cells','ratio cells/heparocytes [6]']))




Total littoral cells: 4.2e+10 SD range 2.8e+10 - 6.2e+10 cells
Number of Kupffer cells is: 3e+10 cells (SD range: 2e+10 - 4e+10 cells)
Number of stellate cells is: 1e+10 cells (SD range: 1e+10 - 2e+10 cells)
_______
Sztark et al., 1986 density derived estimate for the number of stellate cells: 3.1e+10
That overstimate might be due to the small section that was taken just from the parenchymal volume, not calibrated as the results derived from Gates et al., 1981

Estimated stellate cell to hepatocyte ratio: 6.3%
Giampieri et al., 1981 stellate cell to hepatocyte ratio: 6.4%


### Cellular mass of Kupffer and stellate cells
By [Damania et al., 2014](https://www.ncbi.nlm.nih.gov/pubmed/26202404) stellate and Kupffer cells have similar sizes. There is no direct measurements of their mass or volumes, but [Bouwen et al., 1986](https://www.ncbi.nlm.nih.gov/pubmed/3733004),  gives measurements of both diameter and area of profiles of mice Kupffer cells. We combine these with comparison with the volume distriution of liver cell volume that wass assesed in rats [Blouin et al., 1977](http://jcb.rupress.org/content/72/2/441.abstract).

The shape of these cells is said to be stralike, and it difficult to estimate their volume from the diameter. We assume that the shape conversion factor between the volume of a sphere with the maximal diameter and the true volume of the cells, can be estimated by taking the conversion factor of the profile area to a bounding circle, with a power of 3/2.


In [11]:
kupf_sizes = pd.read_excel('Liver_cells_data.xlsx','Kupffer_cells_size',index_col=0,usecols = range(4))

kupf_sizes['2D conv'] =  kupf_sizes['area [µm^2]']/(np.pi*(kupf_sizes['diameter [µm] ']/2)**2)
kupf_sizes['3D conv'] =kupf_sizes['2D conv']**1.5

kupf_sizes['sphere volume'] = 4*np.pi/3*(kupf_sizes['diameter [µm] ']/2)**3

kupf_sizes['volume'] = kupf_sizes['3D conv']*kupf_sizes['sphere volume']


#saving the results of the method based on diameters temporarly to the liver_results
print('Based only on diameter and area measurments:')
for cell in ['Kupffer cells','stellate cells']: 
    liver.res.loc[cell,'cell mass'] = gmean_with_errors(kupf_sizes['volume'])
    total_mass = liver.res.loc[cell,'number'][0]*liver.res.loc[cell,'cell mass'][0]/10**12
    total_mass_unc = propagate_mul_facs([liver.res.loc[cell,'number'][1],liver.res.loc[cell,'cell mass'][1]])
    liver.res.loc[cell,'total cellular mass'] = total_mass,total_mass_unc
    liver.print_params(['cell mass','total cellular mass'],cell)


Based only on diameter and area measurments:
Cell mass of Kupffer cells is: 220 pg (SD range: 136 - 360 pg)
Total cellular mass of Kupffer cells is: 6 grams (SD range: 3 - 11 grams)
Cell mass of stellate cells is: 220 pg (SD range: 136 - 360 pg)
Total cellular mass of stellate cells is: 3 grams (SD range: 2 - 6 grams)


[Sasse et al., 1992](https://www.ncbi.nlm.nih.gov/pubmed/1289084) gives a value of 250 µm3 with no primary reference, similar to the value we got.
However in comparison to [Blouin et al., 1977](http://jcb.rupress.org/content/72/2/441.abstract) it seems to be an underestimate. The sum of the overall mass of them together amount to less than 1% of the total volume of the hepatocyte, in contrast to more than 4% in the measurements in rat (and 1-2% of stellate cells in measured in humans: [Giampieri et al., 1981](https://www.karger.com/Article/PDF/198640), [Sztark et al., 1986](https://www.sciencedirect.com/science/article/pii/S0168827886800472)).


In [12]:
vol_dist =pd.DataFrame(cell_dist_data['% of rat parenchymal vol [1]'])
vol_dist = vol_dist.rename(columns={'% of rat parenchymal vol [1]': 'vol dist'}).drop('endothelial cells')
vol_dist['numbers'] = liver.get_param_vals('number')

vol_dist['predicted total mass'] = vol_dist['vol dist']*liver.res.loc['hepatocytes','total cellular mass'][0]/vol_dist.loc['hepatocytes','vol dist']

vol_dist['predicted cell mass']= 10**12*vol_dist['predicted total mass']/vol_dist['numbers']

#The major uncertianty evolve from thr hepatocytes mass uncertainty which is the major factor in the numbers uncertainty too.  
# vol_dist['predicted cell mass unc'] =  list(zip(*liver_results['number']))[1]
vol_dist['predicted cell mass unc'] = liver.get_param_unc('number')
vol_dist

,vol dist,numbers,predicted total mass,predicted cell mass,predicted cell mass unc
cells,,,,,
hepatocytes,0.8,"230,265,863,085.8",980.0,"4,255.9",1.5
Kupffer cells,0.0,"27,120,607,952.5",26.5,975.4,1.5
stellate cells,0.0,"14,450,412,864.6",17.6,"1,220.4",1.5


We take the geometrical mean of the two estimates (from diameter, and from overall volume distribution):


In [13]:
print('Best estimate, including calibration from volume fraction data')
for cell in ['Kupffer cells','stellate cells']:
    #setting the cell mass we got from the volume fraction estimate with its uncertainty
    cell_mass_from_vols = tuple([vol_dist.loc[cell,'predicted cell mass'],vol_dist.loc[cell,'predicted cell mass unc']])
    #using the utility function 'gmean_with_errors_from_mul' to get the geomtrical mean of the estimates and the uncertainty, taking in acount the current multiplicatino factors 
    liver.res.loc[cell,'cell mass'] =  gmean_with_errors_from_mul([liver.res.loc[cell,'cell mass'],
                                                                   cell_mass_from_vols])

    #using the new cell mass estimate to get a total cellular mass of each of them
    total_mass = liver.res.loc[cell,'number'][0]*liver.res.loc[cell,'cell mass'][0]/10**12
    total_mass_unc = propagate_mul_facs([liver.res.loc[cell,'number'][1],liver.res.loc[cell,'cell mass'][1]])
    #setting the results
    liver.res.loc[cell,'total cellular mass'] = total_mass,total_mass_unc
    liver.print_params(['cell mass','total cellular mass'],cell)


Best estimate, including calibration from volume fraction data
Cell mass of Kupffer cells is: 470 pg (SD range: 230 - 930 pg)
Total cellular mass of Kupffer cells is: 13 grams (SD range: 6 - 28 grams)
Cell mass of stellate cells is: 520 pg (SD range: 250 - 1100 pg)
Total cellular mass of stellate cells is: 8 grams (SD range: 3 - 18 grams)


## Lifespan and cellular turnover
We could find in the literature few sources with measurements of hepatocytes cell cycle in humans ([King et al., 1998](https://www.ncbi.nlm.nih.gov/pubmed/9570240), 
[Werling et al., 2001](https://www.ncbi.nlm.nih.gov/pubmed/11396526), 
[Marshall et al., 2005](https://www.ncbi.nlm.nih.gov/pubmed/15633121), 
[Mun et al., 2006](https://www.ncbi.nlm.nih.gov/pubmed/18376794)).
However there is no concentrated analysis of their results. Moreover, some of the soruces seemed to be of poor quality, and the variation between their results exceeds 2 order of magnitude. Thus we don't provide estimate for the hepatocyte lifespan from human measurements, but only give reference values for lifespan of liver cells taken from measurements in rodents (rats). Those enable an extrapolation of cellular turnover rate, that can serve as an upper bound for human liver cells. 


Lifespan values in rodents were taken from two sources that made use of either labeling technique using H3-thymidine [MacDonald, 1961]() or Metaphase Index using Vinblastine sulfate [Bouwens et al., 1986](https://www.ncbi.nlm.nih.gov/pubmed/3733004). 

In [14]:
rod_lifespans = pd.read_excel('Liver_cells_data.xlsx','Lifespan_rodents',index_col=0,usecols = range(6),skipfooter=3)

#extracting the uncertainties and translating them to multiplication factors: 
rod_ls_unc = uarr_to_mulfac(unp.uarray(rod_lifespans['lifespan in rodent [d]'],
                                       rod_lifespans['lifespan SD in rodent [d]'])).round(2)

#assigning the values to new paramter in the results
liver.res['lifespan in rodents'] = list(zip(rod_lifespans['lifespan in rodent [d]'],rod_ls_unc))


# using the lifespan from rodents to extrapolate potential cellular turnover rates values:
ext_turnover = vol_dist['numbers']/rod_lifespans['lifespan in rodent [d]']
ext_turnover_unc = propagate_mul_facs([liver.get_param_unc('number'),rod_ls_unc]).round(1)

liver.res['extrapolated cellular turnover rate'] = list(zip(ext_turnover,ext_turnover_unc))


ext_mass_turnover = (liver.get_param_vals('total cellular mass')/rod_lifespans['lifespan in rodent [d]']).round(2)
ext_mass_turnover_unc = propagate_mul_facs([liver.get_param_unc('total cellular mass'),rod_ls_unc]).round(1)

liver.res['extrapolated cellular mass turnover rate'] = list(zip(ext_mass_turnover,ext_mass_turnover_unc))

By extrapolation from lifespan measured in rodents (assuming similar lifespan in humans) we can get an upper bound for the cellular turnover rates in humans:

In [15]:
liver.print_params(['lifespan in rodents','extrapolated cellular turnover rate',
                    'extrapolated cellular mass turnover rate'])

hepatocytes:
Lifespan in rodents of hepatocytes is: 360 days (SD range: 310 - 420 days)
Extrapolated cellular turnover rate of hepatocytes is: 6e+08 cells per day (SD range: 4e+08 - 1e+09 cells per day)
Extrapolated cellular mass turnover rate of hepatocytes is: 3 grams per day (SD range: 2 - 4 grams per day)
______________________________
Kupffer cells:
Lifespan in rodents of Kupffer cells is: 400 days (SD range: 190 - 830 days)
Extrapolated cellular turnover rate of Kupffer cells is: 7e+07 cells per day (SD range: 3e+07 - 2e+08 cells per day)
Extrapolated cellular mass turnover rate of Kupffer cells is: 0.0300 grams per day (SD range: 0.0100 - 0.0900 grams per day)
______________________________
stellate cells:
Lifespan in rodents of stellate cells is: 30 days (SD range: 13 - 67 days)
Extrapolated cellular turnover rate of stellate cells is: 5e+08 cells per day (SD range: 2e+08 - 1e+09 cells per day)
Extrapolated cellular mass turnover rate of stellate cells is: 0.25 grams per day (S

## Saving the results to excell

In [16]:
liver.export_to_excel('Summary.xlsx')

no color as Kupffer cells is not in the cell types list
no color as stellate cells is not in the cell types list
